In [5]:
import obi

"""
Define Blocks
"""
simulations_initialize = obi.SimulationsForm.Initialize(
    circuit=obi.Circuit(circuit_path="circuit_1", node_set='hex0'),
    simulation_length=[100, 500]
)
intracellular_location_set_1 = obi.IntracellularLocationSet(circuit=simulations_initialize.circuit, neuron_ids=(1, 2, 3), section='soma')
xyz_extracellular_location_set_1 = obi.XYZExtracellularLocationSet(circuit=simulations_initialize.circuit, xyz_locations=((0.9, 0.2, 0.3), (0.1, 0.1, 0.3)))

id_neuron_set_1 = obi.IDNeuronSet(circuit=simulations_initialize.circuit, neuron_ids=(1, 2, 3))
id_synapse_set_1 = obi.IDSynapseSet(circuit=simulations_initialize.circuit, synapse_ids=(1, 2, 3))

timestamps_1 = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=1, interval=[1.0, 5.0])
stimulus_1 = obi.SynchronousSingleSpikeStimulus(spike_probability=[0.5, 0.8], timestamps=timestamps_1, synapse_set=id_synapse_set_1)
recording_1 = obi.IntracellularLocationSetVoltageRecording(start_time=0.0, end_time=1.0, intracellular_location_set=intracellular_location_set_1)

"""
Fill form with Blocks
"""
simulations_form = obi.SimulationsForm(
                                    initialize=simulations_initialize,

                                    intracellular_location_sets={"id_intracellular_location_set_1": intracellular_location_set_1},
                                    extracellular_location_sets={"xyz_extracellular_location_set_1": xyz_extracellular_location_set_1},

                                    neuron_sets={"id_neuron_set_1": id_neuron_set_1},
                                    synapse_sets={"id_synapse_set_1": id_synapse_set_1},

                                    timestamps={"timestamps_1": timestamps_1}, 
                                    stimuli={"stimulus_1": stimulus_1}, 
                                    recordings={"recording_1": recording_1},
                                    )

"""
Create Scans
"""
grid_scan = obi.GridScan(form=simulations_form, output_root='../obi_output/circuit_simulations/grid_scan')
grid_scan.multiple_value_parameters(display=True)
grid_scan.coordinate_parameters(display=True)
grid_scan.coordinate_instances(display=True)
grid_scan.generate()

print("END")
coupled_scan = obi.CoupledScan(form=simulations_form, output_root='../obi_output/circuit_simulations/coupled_scan')
# # coupled_scan.multiple_value_parameters(display=True)
# # coupled_scan.coordinate_parameters(display=True)
# # coupled_scan.coordinate_instances(display=True)
coupled_scan.generate()

ModuleNotFoundError: No module named 'obi.modeling'